In [211]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer, util

#here we are formatting the data in the json file to be fit as training data

embedmodel = SentenceTransformer('nq-distilbert-base-v1')

questionsraw = []
tagsraw = []
qtpairsraw = []

with open('intents.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    for qanda in data['intents']:
        t = qanda['tag']
        tagsraw.append(t)
        for q in qanda['questions']:
            questionsraw.append(q)
            qtpairsraw.append((q, t)) 
            
tagsraw = sorted(set(tagsonly))

questionsencoded = []
tagsencoded = []

for (q, t) in qtpairsraw:
    question = embedmodel.encode(q)
    questionsencoded.append(question)
    
    tag = tagsraw.index(t)
    tagsencoded.append(tag)
    
questionsencoded = np.array(questionsencoded)
tagsencoded = np.array(tagsencoded)
print(tagsencoded)

[ 8  8  8  8  8  8  8  8  7  7  7 19 19 19  4  4  4  4  4  4  4  4  2  2
  2  2  2  6  6  6  6  6 10 10 10  5  5  5  5  5  0  0  0  0 20 20 20 20
 13 13 13 13 13  9  9  9  9 16 16 16 16 17 17 17 17 17  3  3  3  3 15 15
 15 15  1  1  1 18 18 18 18 11 11 11 12 12 12 14 14 14]


In [206]:
#creating a dataset accessible with data loader so we can iterate over the training data, accessible with indexing

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class ChatBot(Dataset):
    def __init__(self):
        self.n_samples = len(questionsencoded)
        self.x_data = questionsencoded
        self.y_data = tagsencoded
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples
    

class NeuralNet(nn.Module):
    def __init__(self, inputsize, hiddensize, outputsize):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(inputsize, hiddensize)
        self.l2 = nn.Linear(hiddensize, hiddensize)
        self.l3 = nn.Linear(hiddensize, outputsize)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out
    
#hyperparameters
dataset = ChatBot()
batchsize = 8 #can change this
learningrate = 0.001 #can change this
epochs = 1000 #can change this
inputsize = 768
hiddensize = 8
outputsize = len(tagsraw)

loaddata = DataLoader(dataset = dataset, batch_size = batchsize, shuffle=True, num_workers=0)

model = NeuralNet(inputsize, hiddensize, outputsize) 

In [207]:
#loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learningrate)

for epoch in range(epochs):
    for (questionsencoded, tagsencoded) in loaddata:
        
        #forwarding
        outputs = model(questionsencoded)
        loss = criterion(outputs, tagsencoded)
        
        #optimizing
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch + 1) % 100 == 0:
        print(f'epoch {epoch+1}/{epochs}, loss={loss.item():.4f}')
        
print(f'final loss, loss={loss.item():.4f}')

epoch 100/1000, loss=0.0144
epoch 200/1000, loss=0.0055
epoch 300/1000, loss=0.0013
epoch 400/1000, loss=0.0001
epoch 500/1000, loss=0.0002
epoch 600/1000, loss=0.0000
epoch 700/1000, loss=0.0000
epoch 800/1000, loss=0.0000
epoch 900/1000, loss=0.0000
epoch 1000/1000, loss=0.0000
final loss, loss=0.0000


In [208]:
trainingdata = {
    'inputsize': inputsize, 
    'outputsize': outputsize, 
    'hiddensize': hiddensize, 
    'allembeddings': questionsraw, 
    'alltags': tagsraw,
    'state': model.state_dict()
}

newfile = 'new.pth'
torch.save(trainingdata, newfile)

print(f'training complete, file saved to {newfile}')

training complete, file saved to new.pth


In [209]:
with open('intents.json', encoding='utf-8-sig') as file:
    data = json.load(file)
    
training = torch.load(newfile)

inputsize = training['inputsize']
outpusize = training['outputsize']
hiddensize = training['hiddensize']
allembeddings = training['allembeddings']
alltags = training['alltags']
state = training['state']

usemodel = NeuralNet(inputsize, hiddensize, outputsize)
usemodel.load_state_dict(state)
usemodel.eval()

NeuralNet(
  (l1): Linear(in_features=768, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=21, bias=True)
  (relu): ReLU()
)

In [210]:
bot = 'AMAAAA'
print('Ask me anything about alcohol addiction! Type "quit" to exit')

print(tagsencoded)
while True: 
    query = input('You: ')
    if query == 'quit':
        break
    else:
        query = np.array(embedmodel.encode(query))
        query = query.reshape(1, query.shape[0])
        query = torch.from_numpy(query)
        
        output = model(query)
        _, predicted = torch.max(output, dim=0)
        predictedtag = tagsencoded[predicted.item()]
        
        probabilities = torch.softmax(output, dim=1)
        probability = probabilities[0][predicted.item()]
        
        if probability.item() > 0.75:
            for qanda in intents['intents']:
                if predictedtag == qanda['tag']:
                    print(f'{bot}: {qanda["answers"][0]}')
        else:
            print(f'{bot}: Could you rephrase that? I do not understand...')
            


Ask me anything about alcohol addiction! Type "quit" to exit
tensor([13,  6])
You: hey


ValueError: only one element tensors can be converted to Python scalars